In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [2]:
#como el navegador a usar es chrome, hemos descargado ya el driver de la página, guardamos la ruta en una variable
#chrome_path = 'chromedriver_linux64/chromedriver'
#driver = webdriver.Chrome(executable_path = chrome_path)

In [3]:
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/78.0.3904.108 Safari/537.36'}

¿Qué queremos obtener de la página?
1. Nombre del producto
2. Precio (?)
3. Detalles
4. Ingredientes

## 1 OBTENER TODAS LAS URLS DE LOS PRODUCTOS

Para ello, utilizaremos bucles que iteren sobre los elementos de la página.
* Variable global cleansers_url, es el contenedor de todas las urls.
* Los links están dentro de un tag 'a', como sólo nos interesan aquellas que tengan href, pues href=True. 
* Se ha agregado class='product-image" porque si no, el bucle cogía todas las urls (wishlist, url de la imagen etc), y solo nos interesan los productos.
* Append cada uno de los links al contenedor.

In [4]:
masks_url = []

In [5]:
for i in range (1,19):
    url = f'https://www.stylevana.com/en_EU/skincare/masks/gel-masks.html?p={i}' 
    r = requests.get(url)
    soup = BeautifulSoup(r.content, 'lxml')
    masks_list = soup.find_all('div', class_='product-item-info')
    for item in masks_list:
        for link in item.find_all('a', class_='product-image', href=True): 
            masks_url.append(link['href'])
print(len(masks_url))

207


with open("cleansers_url.txt", "w") as output:
    output.write(str(cleansers_url))

In [6]:
len(masks_url)

207

## 2 OBTENER LA INFO DE LA PÁGINA PROPIA DEL PRODUCTO
Info que necesitamos:
* Nombre del producto
* Details
* Ingredients

* Details e Ingredients se encuentran dentro de un grupo <div class='panel-group'

In [9]:
masks_url[206]

'https://www.stylevana.com/en_EU/caolion-blackhead-steam-pore-pack-50g.html'

In [7]:
masks = []

In [10]:
for link in masks_url[0:207]:
    r = requests.get(link, headers=headers) #para cada link en el listado, hacemos una requests
    soup = BeautifulSoup(r.content, 'lxml')
    #usamos try, except porque hay ocasiones en las que no hay info sobre los ingredientes
    try:
        name = soup.find('h1', class_='product-name-h1').text
        details = (soup.find('div', id='product-view-details').text.strip()) # details
        ingredients = soup.find_all('div', id="product-view-ingredients") #ingredients
        for item in ingredients:
            ingredients = item.find('div', class_='panel-body').text.strip()
    except:
        name = 'no info'
        details = 'no info'
        ingredients = 'no info'
        
    masks_info = {
    'name': name,
    'details': details,
    'ingredients': ingredients}
    
    masks.append(masks_info)

In [11]:
print(len(masks))

207


## 3. PASARLO A DATAFRAME

In [12]:
masks_df = pd.DataFrame(masks)
masks_df

,name,details,ingredients
0,Isntree - Real Mugwort Clay Mask - 100ml,Cruelty Free: Yes\n\n\nProduct Type: Wash-off ...,[]
1,I'm From - Honey Mask - 30g,Cruelty Free: Yes\n\n\nProduct Type: Wash-off ...,"Honey, Caprylic/Capric Triglyceride, PEG-7 Gly..."
2,SOME BY MI - Super Matcha Pore Clean Clay Mask...,Product Type: Wash-off Mask\n\n\nSkin Concern:...,[]
3,Isntree - Real Rose Calming Mask - 100ml,Cruelty Free: Yes\n\n\nProduct Type: Gel Mask\...,"Rosa Damascena Flower Water, Glycerin, Water, ..."
4,BE PLAIN - Greenful Bubble Wash-off Mask - 50g,Product Type: Wash-off Mask\n\n\nSkin Concern:...,[]
...,...,...,...
202,HERA - Pink Clay Heating Mask - 50ml,Product Type: Wash-off Mask\n\n\nSkin Concern:...,[]
203,BEAUSTA - O2 Bubble Mask - 20ml,Product Type: Wash-off Mask\n\n\nProduct Unit:...,[]
204,BIO-ESSENCE - Bio-Bounce Collagen Night Mask -...,Product Type: Sleeping Mask\n\n\nProduct Unit:...,[]
205,DR. JOU - Six Essence Hyaluronic Moisturizing ...,Product Type: Gel Mask\n\n\nSkin Concern: Anti...,[]


In [13]:
masks = masks_df.to_csv('data/masks.csv')